In [5]:
import pickle
import os
import tensorflow as tf
import matplotlib.pyplot as plt
# import mpl_toolkits.mplot3d as Axes3D

from database import *

%matplotlib inline

# Datasets

Generate training, testing, and validation datasets

In [4]:
# Import data from database
step = 1/10e3  # microphone sample period, sec
mic_list = ("Ambient", "Mic 0", "Mic 1", "Mic 2", "Mic 3")  # for setting the legend
fs = 1/step  # sample rate, Hz

# Load data from the database
# host = "mysql.ecn.purdue.edu"  # 128.46.154.164
# user = "op_point_test"
# database = "op_point_test"
host = 'localhost'
user = 'root'
database = 'mysql'
table_name = '100_op_point_test'
password = 'admin'
# with open("password.txt", "r") as f:
#   password = f.read().rstrip()
eng = connect_to_db(host, user, password, database)
tic = timeit.default_timer()
data = import_data(eng, table_name)
toc = timeit.default_timer()
if eng.open:
    eng.close()
print("Elapsed time: {} sec".format(toc-tic))

Number of rows: 42331
Keys: ['dynamicP', 'opPointDes', 'staticP', 'atmosphericP', 'flameStatus', 'temperature', 'opPointAct', 'dateTimeStamp']
Loading data ................................................... done!
Elapsed time: 42.98329856200144 sec


In [6]:
# Import processed data from pickle
processed_data = []
for mic in mic_list:
    fname = os.path.join('.', 'Processed', 'short_fft_waterfall_{}.pickle'.format(mic))
    processed_data.append(pickle.load(open(fname, 'rb')))

In [7]:
dataset = []
labelset = []
for num in range(data['opPointAct'].shape[0]):
    start = num*100
    vec = np.concatenate((data['flameStatus'][num], data['opPointAct'][num], data['temperature'][num],
                          data['staticP'][num],
                          np.concatenate([row for row in data['dynamicP'][:, start:start+100]])), axis=0)
    dataset.append(vec)
    vec = np.concatenate((data['flameStatus'][num],
                          np.concatenate([row['res'][num,:] for row in processed_data])), axis=0)
    labelset.append(vec)
dataset = np.array(dataset)
labelset = np.array(labelset)
dataset.shape, labelset.shape

((42332, 511), (42332, 501))

In [8]:
input_size = dataset.shape[1]
output_size = labelset.shape[1]
num_samples = dataset.shape[0]
batch_size = 64
num_unrollings = 10

In [9]:
class BatchGenerator(object):
    def __init__(self, data, batch_size, num_unrollings):
        self._data = data
        self._num_samples = data.shape[0]
        self._batch_size = batch_size
        self._num_unrollings = num_unrollings
        segment = self._num_samples // batch_size
        self._offset = 0
        self._last_batch = self._next_batch()

    def _next_batch(self):
        """Generate a single batch from the current cursor position in the data."""
        batch = self._data[self._offset:self._offset+self._batch_size, :]
        self._offset += self._batch_size
        if self._num_samples - self._offset < self._batch_size:
            self._offset = 0
        return batch

    def next(self):
        """Generate the next array of batches from the data. The array consists of
        the last batch of the previous array, followed by num_unrollings new ones.
        """
        batches = [self._last_batch]
        for step in range(self._num_unrollings):
            batches.append(self._next_batch())
        self._last_batch = batches[-1]
        return batches

In [10]:
train_size = np.int(0.85*num_samples)
valid_size = len(dataset) - train_size
train_batches = BatchGenerator(dataset[:train_size,:], batch_size, num_unrollings)
train_label_batches = BatchGenerator(labelset[:train_size,:], batch_size, num_unrollings)
valid_batches = BatchGenerator(dataset[train_size:,:], 1, 1)
valid_label_batches = BatchGenerator(labelset[train_size:,:], 1, 1)

In [11]:
def logprob(predictions, labels):
    """Log-probability of the true labels in a predicted batch."""
    predictions[predictions < 1e-10] = 1e-10
    return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

# LSTM RNN Definition

Define the long short-term memory (LSTM) recurrent neural network (RNN). Borrowing heavily from the Deep Learning Udacity course.

In [18]:
num_nodes = 64  #TODO tune this!
num_steps = 1001

graph = tf.Graph()
with graph.as_default():
    # Parameters
    # Combined input, forget, memory cell, and output weights for input and previous state. Also biases.
    ifcox = tf.Variable(tf.truncated_normal([input_size, 4*num_nodes], 0.0, 1/np.sqrt(input_size)))
    ifcom = tf.Variable(tf.truncated_normal([num_nodes, 4*num_nodes], 0.0, 1/np.sqrt(input_size)))
    ifcob = tf.Variable(tf.zeros([1, 4*num_nodes]))
    # Variables saving state across unrollings.
    saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    # Classifier weights and biases.
    w = tf.Variable(tf.truncated_normal([num_nodes, output_size], 0.0, 1/np.sqrt(output_size)))
    b = tf.Variable(tf.zeros([output_size]))
    
    # Create saver
    saver = tf.train.Saver()
    
    # Definition of the cell computation
    def lstm_cell(i, o, state):
        """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
        Note that in this formulation, we omit the various connections between the
        previous state and the gates."""
        combined = tf.matmul(i, ifcox) + tf.matmul(o, ifcom) + ifcob
        input_gate = tf.sigmoid(combined[:, :num_nodes])
        forget_gate = tf.sigmoid(combined[:, num_nodes:2*num_nodes])
        update = combined[:, 2*num_nodes:3*num_nodes]
        state = forget_gate * state + input_gate * tf.tanh(update)
        output_gate = tf.sigmoid(combined[:, 3*num_nodes:])
        return output_gate * tf.tanh(state), state
    
    # Input data
    train_data = list()
    train_labels = list()
    for _ in range(num_unrollings + 1):
        train_data.append(tf.placeholder(tf.float32, shape=[batch_size, input_size]))
        train_labels.append(tf.placeholder(tf.float32, shape=[batch_size, output_size]))
    train_inputs = train_data[:num_unrollings]
    train_outputs = train_labels[1:]  # labels are shifted by one time step.
    
    # Unrolled LSTM loop
    outputs = list()
    output = saved_output
    state = saved_state
    for i in train_inputs:
        output, state = lstm_cell(i, output, state)
        outputs.append(output)
    
    # Saving state across unrollings
    with tf.control_dependencies([saved_output.assign(output),
                                  saved_state.assign(state)]):
        # Classifier.
        logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits,
                tf.concat(0, train_outputs)))
    
    # Optimizer
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(10.0, global_step, 2*num_steps//3, 0.1, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)
    
    # Predictions
    train_prediction = tf.nn.softmax(logits)
    
    # Sampling and validation eval: batch 1, no unrolling.
    sample_input = tf.placeholder(tf.float32, shape=[1, input_size])
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    reset_sample_state = tf.group(saved_sample_output.assign(tf.zeros([1, num_nodes])),
            saved_sample_state.assign(tf.zeros([1, num_nodes])))
    sample_output, sample_state = lstm_cell(sample_input, saved_sample_output,
            saved_sample_state)
    with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                  saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

# Run

Train the network on the dataset and observe the performance

In [20]:
summary_frequency = 100
mean_loss = 0

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized.')
    for step in range(num_steps):
        # Generate next training batch
        batches = train_batches.next()
        labels = train_label_batches.next()
        
        # Generate feed_dict
        feed_dict = dict()
        for i in range(num_unrollings + 1):
            feed_dict[train_data[i]] = batches[i]
            feed_dict[train_labels[i]] = labels[i]
        _, l, predictions, lr = session.run(
                [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
        mean_loss += l
        
        # Update to the user periodically
        if step % summary_frequency == 0:
            # Output some information about our training performance
            if step > 0:
                mean_loss = mean_loss / summary_frequency
            # The mean loss is an estimate of the loss over the last few batches.
            print('Average loss at step {}: {} learning rate: {}'.format(step, mean_loss, lr))
            mean_loss = 0
            print('Minibatch perplexity: {}'.format(float(np.exp(logprob(predictions,
                    np.concatenate(labels[1:], axis=0))))))

    # Measure validation set perplexity.
    reset_sample_state.run()
    valid_logprob = 0
    for num in range(valid_size):
        b = valid_batches.next()
        l = valid_label_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, l[1])
    print('Validation set perplexity: {}'.format(float(np.exp(valid_logprob / valid_size))))
    
    save_path = saver.save(session, './tmp/model.ckpt')
    print('Model saved in file: {}'.format(save_path))

Initialized.
Average loss at step 0: 6.200928688049316 learning rate: 10.0
Minibatch perplexity: 493.20673794490546
Average loss at step 100: 0.007128275972049441 learning rate: 10.0
Minibatch perplexity: 1.0012317684903522
Average loss at step 200: 0.0031575521343620494 learning rate: 10.0
Minibatch perplexity: 1.0005054479820132
Average loss at step 300: 0.0027715624694246797 learning rate: 10.0
Minibatch perplexity: 1.0016356704979006
Average loss at step 400: 0.0031859814579365776 learning rate: 10.0
Minibatch perplexity: 1.001267044307595
Average loss at step 500: 0.0027533012168714775 learning rate: 10.0
Minibatch perplexity: 1.0173151158684255
Average loss at step 600: 0.0029846351256128402 learning rate: 10.0
Minibatch perplexity: 1.0011311792758943
Average loss at step 700: 0.003126926047843881 learning rate: 1.0
Minibatch perplexity: 1.0013175536929602
Average loss at step 800: 0.0027722151251509787 learning rate: 1.0
Minibatch perplexity: 1.0012982689702687
Average loss at s

# Analyze

Analyze the performance and see what we can do with it

In [21]:
#TODO
with tf.Session(graph=graph) as session:
    saver.restore(session, './tmp/model.ckpt')
    print('Model restored.')
    reset_sample_state.run()
    valid_logprob = 0
    for num in range(10):
        b = valid_batches.next()
        l = valid_label_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = logprob(predictions, l[1])
        print('Logprob prediction accuracy: {}'.format(np.exp(valid_logprob)))

Model restored.
Logprob prediction accuracy: 1.0114345435720058
Logprob prediction accuracy: 1.0017291618745077
Logprob prediction accuracy: 1.001293345008001
Logprob prediction accuracy: 1.0012435861506115
Logprob prediction accuracy: 1.001236654929073
Logprob prediction accuracy: 1.0012364158655769
Logprob prediction accuracy: 1.0012364158655256
Logprob prediction accuracy: 1.0012364158657951
Logprob prediction accuracy: 1.0012364159959932
Logprob prediction accuracy: 1.0012364158655291
